In [ ]:
import geopandas as gpd
import pandas as pd
import polars as pl
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from dotenv import load_dotenv
import os
import sys
# To ensure the function can be import
if os.path.dirname(os.getcwd()) not in sys.path:
    sys.path.append(os.path.dirname(os.getcwd()))
# Import own function packages
import spm
# To set the environement to get access to password protected file
load_dotenv()
# Set the mapbox token to access for mapbox map
px.set_mapbox_access_token(os.getenv("MAPBOX_TOKEN"))

# To prepare SPM servic providing GP file
gp_df = pd.read_excel(spm.file._gp_file)

# To calculate the area per polygon
district = gpd.read_file(spm.file._map_district)
district = spm.map.get_polygon_area(district)

# Prepare the parliament polygon file
parlimen = gpd.read_file(spm.file._map_parlimen)

# Prepare the ascii file
population_ascii = spm.map.convert_pandas_geopandas(pd.read_csv(spm.file._population_ascii),lat="Y",lon="X")

# Spatial join both population
temp_population = population_ascii.sjoin(parlimen, how="inner")\
                                  .drop(columns=["state", "index_right", "code_state"])\
                                  .sjoin(district, how="inner")\
                                  .drop(columns=["index_right", "geometry"])

# Count the total population per parliament
temp_population = temp_population.merge(
    temp_population.pivot_table(index="code_parlimen", values="Z", aggfunc=sum)\
                   .reset_index().rename(columns={"Z":"parlimen_z"}),
    how="left", on="code_parlimen")

# Match each point of population with nearest GP (Take long time to run)
# population_gp = spm.gp.match_nearest(df1=temp_population, df1_lat="Y", df1_lon="X",
#                                  df2=gp_df.loc[:,("id", "clinic_name", "address", "Latitude", "Longitude")], 
#                                  df2_lat="Latitude", df2_lon="Longitude",
#                                  column_to_match="id")

# # Get the population count per parliament
# str_parlimen = pl.read_parquet(spm._file._spm_full_parquet)\
#                  .unique(subset="id_ben")\
#                  .group_by("code_parlimen").len("str_count").to_pandas()

# # Left join both of the dataframe
# population_gp = population_gp.merge(str_parlimen, how="left", on="code_parlimen")

# # Divide the estimated_str with total population per parliament time population per point
# population_gp.loc[:,"estimated_str"] = population_gp.loc[:,"str_count"] * population_gp.loc[:,"Z"] / population_gp.loc[:,"parlimen_z"]

# # To check on dataframe
# population_gp
temp_population

In [ ]:
# Read the population_ascii that have merged with data from ProtectHealth
population_ascii = pl.read_parquet(spm.file._population_str_ascii_gp_households)

# Read the file that contain gp information
gp_df = pl.read_excel(spm.file._gp_file)

# Read the district file # 'https://storage.dosm.gov.my/population/population_district.parquet'
district_population = pl.read_parquet(spm.file._population_district)\
                        .filter(pl.col("district").is_in(spm.provider._district_name_list),
                                pl.col("date").cast(pl.String) == "2022-01-01")

# Read HIES 'https://storage.dosm.gov.my/hies/hies_district.parquet'
hies_df = pl.read_parquet(spm.file._hies_district)

# Create a merge to get hies first
merged_df = district_population.filter(pl.col("sex") == "both",
                                       pl.col("age") == "overall",
                                       pl.col("ethnicity") == "overall")\
                               .join(hies_df, on=["date", "state", "district"])\
                               .drop("date", "sex", "age", "ethnicity")\
                               .join(population_ascii.group_by("district").agg(pl.col("estimated_str").sum()), on="district")\
                               .join(gp_df.group_by("district").len("gp_num"), on="district")\
                               .with_columns_seq((pl.col("poverty") * 10 * pl.col("population")).alias("estimated_poverty"),
                                                 (pl.col("estimated_str")/pl.col("gp_num")).alias("gp_str_ratio"),
                                                 (pl.col("estimated_str")/pl.col("population")).alias("str_pop_percentage"))
merged_df = merged_df.with_columns((pl.col("estimated_poverty")/pl.col("gp_num")).alias("gp_pov_ratio"))

print(merged_df.to_pandas().round(2).to_markdown(tablefmt="pretty"))

In [ ]:
# Read both active SPM providing GP and all private GP that registered under the private medical practice division
gp_df = pl.read_excel(spm.file._gp_file)
all_gp_df = pl.read_excel(spm.file._full_gp_file, sheet_name="gp_list")

# Pivot both spm active provding gp and total private gp registered
all_gp_pt = all_gp_df.group_by("district").len("registered")\
                     .with_columns((pl.col("registered")/pl.col("registered").sum()).alias("Percentage for Total Private GP"))
gp_pt = gp_df.group_by("district").len("spm_gp")\
             .with_columns((pl.col("spm_gp")/pl.col("spm_gp").sum()).alias("Percentage of Active SPM Service Providing GPs"))

# Join both the pivoted table on district and calculate the percentage of gp registered compare to whole district.
gp_registerd=gp_pt.join(all_gp_pt, how="left", on="district")\
           .with_columns((pl.col("spm_gp")/pl.col("registered")).alias("Active SPM Service Providing GP/ Total GP"))
print(gp_registerd)

# Change to pandas for next step
gp_df = gp_df.to_pandas()

# To roughly know the distribution of private gp that is not providing SPM services
exclusive_gp = all_gp_df.to_pandas()\
    .query(f"code_state_district.isin({spm.provider._district_code_list}) & distance_km > 1")\

# To allow user to know what map_box style available
print(spm.map._mapbox_style)

# To draw the polygon of district
fig = spm.map.draw_polygon_line(district, polygon_name="district")\
    .update_layout(showlegend=False)

# Scatter plot active SPM service providing GP with green color spot
gp_fig = go.Figure(go.Scattermapbox(
    mode = "markers",
    lon = gp_df["Longitude"], lat = gp_df["Latitude"],
    marker = {'size':10, 'symbol': "circle", "color":"red"},
    hovertext = gp_df["clinic_name"],
    ))

# Scatter plot private GP not enrolled SPM
add_gp = go.Figure(go.Scattermapbox(
    mode = "markers+text",
    lon = exclusive_gp["Longitude"], lat = exclusive_gp["Latitude"],
    marker = {'size':10, 'symbol': "circle", "color":"blue"},
    hovertext = exclusive_gp["clinic_name"],
    ))

# Plot 3 figures together
fig.add_trace(gp_fig.data[0])\
    .update_layout(mapbox = {
        'accesstoken': os.getenv("MAPBOX_TOKEN"),
        'style': spm.map._mapbox_style[5], 'zoom': 5, "center":spm.map._map_center},
        margin={"r":0,"t":0,"l":0,"b":0})

In [ ]:
# Load the population ascii from ProtectHealth
population_ascii = pl.read_parquet(spm.file._population_str_ascii_gp_households) 

# Read the SPM service providing GP file
gp_df = pl.read_excel(spm.file._gp_file)

# Read the district file # 'https://storage.dosm.gov.my/population/population_district.parquet'
district_population = pl.read_parquet(spm.file._population_district)\
                        .filter(pl.col("district").is_in(spm.provider._district_name_list),
                                pl.col("date").cast(pl.String) == "2022-01-01")

# Filter the target population
target_population = population_ascii.filter(pl.col("code_state_district").is_in(spm.provider._district_code_list))

# Read HIES 'https://storage.dosm.gov.my/hies/hies_district.parquet'
hies_df = pl.read_parquet(spm.file._hies_district)

# Create a merge to get hies first
merged_df = district_population.filter(pl.col("sex") == "both",
                                       pl.col("age") == "overall",
                                       pl.col("ethnicity") == "overall")\
                               .join(hies_df, on=["date", "state", "district"])\
                               .drop("date", "sex", "age", "ethnicity")\
                               .join(population_ascii.group_by("district").agg(pl.col("estimated_str").sum()), on="district")\
                               .join(gp_df.group_by("district").len("gp_num"), on="district")\
                               .with_columns_seq((pl.col("poverty") * 10 * pl.col("population")).alias("estimated_poverty"),
                                                 (pl.col("estimated_str")/pl.col("gp_num")).alias("gp_str_ratio"),
                                                 (pl.col("estimated_str")/pl.col("population")).alias("str_pop_percentage"))
merged_df = merged_df.with_columns((pl.col("estimated_poverty")/pl.col("gp_num")).alias("gp_pov_ratio"))

print(merged_df.to_pandas().round(2).to_markdown(tablefmt="pretty"))

# Convert population_ascii to pandas dataframe
population_ascii = population_ascii.to_pandas()

# Chorepleth for str within each parlimen
spm.map.draw_chorepleth(map_file=spm.file._map_parlimen,
                    df= population_ascii.pivot_table(index="code_parlimen",
                                                     values="estimated_str",
                                                     aggfunc=sum)\
                                        .reset_index(),
                    location="code_parlimen",
                    z="estimated_str",
                    featureidkey="code_parlimen",
                    colorscale="Viridis_r").show()

# Chorepleth for str within each district
spm.map.draw_chorepleth(map_file=spm.file._map_district,
                    df= population_ascii.pivot_table(index="code_state_district",
                                                     values="estimated_str",
                                                     aggfunc=sum)\
                                        .reset_index(),
                    location="code_state_district",
                    z="estimated_str",
                    featureidkey="code_state_district",
                    colorscale="Viridis_r").show()

# In view of density map not properly view, using scatter plot on STR population density
px.scatter_mapbox(population_ascii, 
                  lat="Y", lon="X",
                  size="estimated_str", 
                  color="state",
                  color_continuous_scale="blues",
                  mapbox_style="basic",
                  zoom=5,
                  center=spm.map._map_center)\
  .update_layout(margin={"r":0,"t":0,"l":0,"b":0})

In [ ]:
# Prepare the bins and labels to categorize distance_km
bin_edges = list(range(0, int(target_population["distance_km"].max()) + 5, 5))
bins_labels = [f"{start}-{end}" for start, end in zip(bin_edges[:-1], bin_edges[1:])]
target_population = target_population.with_columns(pl.col("distance_km").cut(breaks=bin_edges[1:-1], labels=bins_labels)\
                                 .alias("distance_cat"))

# Pivot the distance for all the estimated_str
distance_pt = target_population.group_by("distance_cat").agg(pl.col("estimated_str").sum())\
                 .with_columns(pl.col("estimated_str")/1000,
                               (pl.col("estimated_str")/pl.col("estimated_str").sum() * 100).alias("percentage"))\
                 .sort(pl.col("estimated_str"), descending=True)

# Print the information
print(distance_pt.to_pandas().round(2).to_markdown(tablefmt="pretty"))

# Pivot the distance according to district
district_distance_pt = target_population.pivot(on="district", 
                                               index="distance_cat", 
                                               values="estimated_str", 
                                               aggregate_function="sum")

# Show the above information 
print(district_distance_pt.to_pandas().round(2).to_markdown(tablefmt="pretty"))

# Plot the bar chart for distance in categorical order
px.histogram(distance_pt, 
             x="distance_cat", y="estimated_str", 
             histnorm="probability",
             text_auto=".2%",
             title="Distribution of Distance Between STR Population and Their Nearest Active SPM Service Providers (km)").show()

# Boxplot for all estimated_str
px.box(target_population,
       height=600,
       x="distance_km",
       title="Boxplot of Distance Between STR Population and Their Nearest Active SPM Service Providers (km)").show()

# Bar chart for each district
px.histogram(target_population, 
             x="distance_cat", y="estimated_str",
             barmode="group",
             category_orders={"distance_cat":bins_labels},
             color="district",
             histnorm="probability",
             log_y=True,
             text_auto=".2%",
             title="Distribution of Distance Between STR Population In Each District and Their Nearest Active SPM Service Providers (km)").show()

# Boxplot for each district
px.box(target_population,
       color="district",
       y="distance_km",
       title="Boxplot of Distance Between STR Population In Each District and Their Nearest Active SPM Service Providers (km)").show()

In [ ]:
# Create all 10 districts ann descriptive analyssis
all_ann = spm.population.ann(
    df=target_population,
    n_column="estimated_str",
    a_column="area",
    distance_column="distance_km",
    district_column= "district")

# Loop through each districts for ann analysis
for district in spm.provider._district_name_list:
    all_ann = pl.concat([all_ann,
                         spm.population.ann(
                                 df = target_population.filter(pl.col("district") == district),
                                 n_column="estimated_str",
                                 a_column="area",
                                 distance_column="distance_km",
                                 district_column= "district"
                             )])

# Display the information
print(all_ann.sort(pl.col("ANN"), descending=True).to_pandas().round(2).to_markdown(tablefmt="pretty"))
# all_ann.sort(pl.col("ANN"), descending=True).to_pandas().to_clipboard(index=False)

In [ ]:
import plotly.figure_factory as ff
# Overlay analysis
population_fig = ff.create_hexbin_mapbox(
            data_frame=target_population.select("X", "Y", "estimated_str"), 
            lat="Y", 
            lon="X",
            color = "estimated_str",
            color_continuous_scale = "mint",
            agg_func=np.sum,
            nx_hexagon=1000, 
            opacity=0.3, 
            min_count=2, 
            show_original_data=False,
            title="Overlay Analysis Between STR Recpieints Population and Active SPM Service Providing GPs"
        )

# It wont show
# population_fig = px.scatter_mapbox(target_population, lat="lat", lon="lon", size="estimated_str", color="district")

gp_fig = go.Figure(go.Scattermapbox(
    mode = "markers+text",
    lon = gp_df["Longitude"], lat = gp_df["Latitude"],
    marker = {'size':10, 'symbol': "circle", "color":"green"},
    hovertext = gp_df["clinic_name"],
    ))

add_gp = go.Figure(go.Scattermapbox(
    mode = "markers+text",
    lon = exclusive_gp["Longitude"], lat = exclusive_gp["Latitude"],
    marker = {'size':10, 'symbol': "circle", "color":"red"},
    hovertext = exclusive_gp["clinic_name"],
    ))

gp_fig.add_trace(population_fig.data[0])
gp_fig.add_trace(add_gp.data[0])

gp_fig.update_layout(
    mapbox = {
        'accesstoken': os.getenv("MAPBOX_TOKEN"),
        'style': "basic", 'zoom': 5, "center":spm.map._map_center},
    showlegend = False, height=800,
    ).update_layout(margin={"r":0,"t":0,"l":0,"b":0})

gp_fig.show()

In [ ]:
from pypdf import PdfWriter

merger = PdfWriter()

for pdf in ["data/2024_08_28_spm_research.pdf",
            "data/appendix_c.pdf",
            "data/mrec_approval_english.pdf",
            "data/turnitin_bw.pdf"]:
    merger.append(pdf)

for page in merger.pages:
    page.compress_content_streams()  # This is CPU intensive!

for page in merger.pages:
    for img in page.images:
        img.replace(img.image, quality=80)

merger.write("data/22110451_correction.pdf")
merger.close()

In [6]:
import geopandas as gpd
import pandas as pd
import polars as pl
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from dotenv import load_dotenv
import os
import sys
# To ensure the function can be import
if os.path.dirname(os.getcwd()) not in sys.path:
    sys.path.append(os.path.dirname(os.getcwd()))
# Import own function packages
import spm
# To set the environement to get access to password protected file
load_dotenv()

df = pl.read_parquet(spm.file._population_str_ascii_gp_households)
df.join(df.group_by("district").agg((pl.col("Z").sum()).alias("district_z")),
        how="left",on="district").drop(["str_count", "estimated_str"])\
            .write_parquet(f"data/information/ascii_gp_district_parlimen.parquet", use_pyarrow=True)